In [ ]:
from arango import ArangoClient
import cugraph
import cudf
import networkx as nx

In [ ]:
def find_influential_connections(node_key, db_name="graph_db", graph_name="ai_incidents_graph"):
    """
    Find the most influential connections of a given node using AQL and cuGraph.
    """
    try:
        # Initialize the ArangoDB client
        client = ArangoClient(hosts="http://localhost:8529")

        # Connect to the database
        db = client.db(db_name, username="root", password="passwd")

        # Define the AQL query to get the subgraph around the node
        query = f"""
        FOR v, e IN 1..2 ANY 'nodes/{node_key}' edges
        RETURN {{ node: v, edge: e }}
        """

        # Execute the query
        cursor = db.aql.execute(query)
        results = [doc for doc in cursor]

        # Convert the results to a NetworkX graph
        G = nx.Graph()
        for result in results:
            G.add_edge(result["node"]["_key"], result["edge"]["_to"])

        # Compute PageRank for the subgraph
        edges = [(u, v) for u, v in G.edges()]
        df = cudf.DataFrame(edges, columns=["src", "dst"])
        g = cugraph.Graph()
        g.from_cudf_edgelist(df, source="src", destination="dst")
        pagerank_df = cugraph.pagerank(g)

        # Sort by PageRank score
        pagerank_df = pagerank_df.sort_values("pagerank", ascending=False)

        return pagerank_df

    except Exception as e:
        print(f"Error executing query: {e}")
        return []

In [ ]:
# Example usage
if __name__ == "__main__":
    node_key = "Content_Recommendation_0"
    influential_connections = find_influential_connections(node_key)
    print(f"Most influential connections of '{node_key}':")
    print(influential_connections)